# Algoritmos de Classificação: Regressão Logística

![image.png](https://somostera.com/_nuxt/img/7eecdb7.svg)

## Warm Up

![image.png](https://media.giphy.com/media/l0Ex47BWhZ7bsaKcg/giphy.gif)

## Expectativas!

O que vocês esperam da aula de hoje?

![image.png](https://img.buzzfeed.com/buzzfeed-static/static/enhanced/web04/2012/2/14/17/enhanced-buzz-8358-1329258536-95.jpg?downsize=800:*&output-format=auto&output-quality=auto)

## Objetivo
> Avaliar se existem fatores que influenciam na promoção dos funcionários

### Contexto
> O RH da empresa está se tornando mais seletivo ao contratar novos funcionários. Dessa forma, ele quer entender o perfil de promoção dos funcionários para que no processo de seleção algum desses fatores sejam considerados. Além disso, um modelo de predição poderia ajudar na definição do budget de promoção. Você faz parte da área de **Data Science** dessa empresa, e precisa encontrar alguns insights que subsidiem o RH.

## Conclusão
> TBD

### Updated at
> 30/Jan/2020 by Tera-DSC Team

### Dataset
> Dados históricos dos funcionários da empresa.

### Warning
> Dados sensíveis.

# **Regressão** ???

![image.png](https://media.giphy.com/media/NsuQixwGTHy8M/giphy.gif)

In [ ]:
"""












"""

> O modelo de regressão logística é semelhante ao modelo de regressão linear. No entanto, no modelo logístico a variável resposta $ Y_i $ é binária. Uma variável binária assume dois valores, como por exemplo, $ Y_i=0 $ e $ Y_i=1, $ denominados "fracasso" e "sucesso", respectivamente. Neste caso, "sucesso" é o evento de interesse.

## Diferença entre Distribuição Linear e Logística
![image.png](https://estatsite.files.wordpress.com/2018/08/linear_vs_logistic_regression.jpg)

In [ ]:
#-- importando as libs
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

from statsmodels.stats.outliers_influence import variance_inflation_factor
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.metrics import (accuracy_score, confusion_matrix, classification_report,
                             roc_auc_score, log_loss, plot_confusion_matrix, roc_curve, auc, plot_roc_curve)

plt.rcParams["figure.figsize"] = (22,15)

In [ ]:
#-- setando parâmetros de font
sns.set_context("notebook", font_scale=1.5)

In [ ]:
#-- carregando o dataset
df = pd.read_excel('../data/Base Analytics.xlsx', sheet_name='Censo sem Estagiário')
df.head().T

## Exploratory Data Analysis (EDA)

In [ ]:
#-- apresentando as dimensões do dataset
df.shape

In [ ]:
#-- printando o tipo das variáveis
df.dtypes

In [ ]:
#-- printando os nomes das colunas
df.info()

In [ ]:
#-- pritnando a quantidade de missing values
null_count = df.isnull().sum().sort_values(ascending=False)
null_percentage = null_count / len(df)
pd.DataFrame(data=[null_count, null_percentage],
             index=['null_amount', 'null_ratio']).T

In [ ]:
#-- printando algumas métricas dos dados numéricos do dataset
df.describe()

## Será que eu posso utilizar todo o Dataset?

In [ ]:
"""














"""

In [ ]:
#-- retornando menor data de admissão
min(df['Admissão'])

In [ ]:
#-- filtrando pela data de admissão
df_ano = df[df['Admissão'] >= '2011-01-01']

In [ ]:
#-- retornando o tamanho do dataset
df_ano.shape

In [ ]:
#-- verificando os missing values do dataset
null_count = df_ano.isnull().sum().sort_values(ascending=False)
null_percentage = null_count / len(df_ano)
pd.DataFrame(data=[null_count, null_percentage],
             index=['null_amount', 'null_ratio']).T

### Distribuição das Idades

In [ ]:
#-- pritando um histograma da idade de admissão dos funcionários
_ = sns.distplot(df_ano['Idade na Admissão'])

### Hora Extra

In [ ]:
#-- plotando o gráfico das horas extras
_ = sns.distplot(df_ano['Hora Extra 2016'])

In [ ]:
#-- preenchendo os missing values com flags
df_ano['Hora Extra 2016'].fillna(-100, inplace = True)

In [ ]:
#-- plotando o hostograma das horas extras
_ = sns.distplot(df_ano['Hora Extra 2016'])

### Qual a idade de desligamento?

In [ ]:
#-- plotando o histograma da idade de desligamento
_ = sns.distplot(df_ano['Idade Atual/ Desligamento'])

### Qual outros gráficos vocês fariam?

In [ ]:
_ = sns.distplot(df_ano['Tempo de Casa'])

In [ ]:
#-- plotando boxplot da promoção pelo tempo de casa
_ = sns.boxplot(x=df_ano['PROMOVIDO'], y=df_ano['Tempo de Casa'])

## Preparando os dados

In [ ]:
#-- separando a variável do dataset de análise
X = df_ano.drop(columns=['PROMOVIDO'])
y = df_ano['PROMOVIDO']

In [ ]:
#-- separando os dados para treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=0)

## Aplicando a Regressão Logística

In [ ]:
#-- guardando o algoritmo de regressão logística
logreg = LogisticRegression()

In [ ]:
#-- aplicando o algoritmo 
logreg.fit(X_train, y_train)

#-- predizendo as respostas
y_pred = logreg.predict(X_test)

In [ ]:
#-- selecionadndo apenas as variáveis númericas
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_ano.select_dtypes(include=numerics).T

In [ ]:
#-- criando um dataset novo
df_ano_numeric = df_ano.select_dtypes(include=numerics).copy()

In [ ]:
#-- selecionando as colunas desse novo dataset
df_ano_numeric = df_ano_numeric.drop(columns=['ADP', 'Cod.Cargo', 'Cod.Cargo Admissão', 'CC', 'Hora Extra 2016', 'Hora Negativa 2016', 'Ad. Noturno 2016', 'Absenteísmo 2016', 'Hora Extra 2017', 'Hora Negativa 2017', 'Ad. Noturno 2017', 'Absenteísmo 2017', 'Banda', '2012/13 Goal Achievement'], axis=1)

In [ ]:
#-- pritando o head do novo dataset
df_ano_numeric.head().T

In [ ]:
#-- verificando os missing values do dataset
null_count = df_ano_numeric.isnull().sum().sort_values(ascending=False)
null_percentage = null_count / len(df_ano_numeric)
pd.DataFrame(data=[null_count, null_percentage],
             index=['null_amount', 'null_ratio']).T

In [ ]:
#-- removendo as colunas de 2013/14 e 2014/15
df_ano_numeric.drop(columns=['2013/14 Goal Achievement', '2014/15 Goal Achievement'], inplace = True)

In [ ]:
#-- preenchendo os missing values
df_ano_numeric.fillna(method='bfill', inplace=True)
df_ano_numeric.fillna(method='ffill', inplace=True)

In [ ]:
#-- verificando se os missing values do dataset foram todas preenchidos
null_count = df_ano_numeric.isnull().sum().sort_values(ascending=False)
null_percentage = null_count / len(df_ano_numeric)
pd.DataFrame(data=[null_count, null_percentage],
             index=['null_amount', 'null_ratio']).T

In [ ]:
#-- printando as dimensões
df_ano_numeric.shape

In [ ]:
#-- verificando os tipos das variáveis
df_ano_numeric.info()

In [ ]:
#-- criando os datasets para análise
X = df_ano_numeric.drop(columns=['PROMOVIDO'])
y = df_ano_numeric['PROMOVIDO']

In [ ]:
#-- separando a parte de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=0)

In [ ]:
#-- apllicando o algoritmo
logreg.fit(X_train, y_train)
#-- calculando a predição
y_pred = logreg.predict(X_test)

In [ ]:
#-- printando os cinco primeiros valores da predição
y_pred[:5]

In [ ]:
#-- pritando a acuracia do teste
accuracy_test = accuracy_score(y_test, y_pred)
print(f'accuracy (test): {accuracy_test*100:.1f}%')

## Avaliação do modelo

### Matriz de Confusão, Precisão, Recall

In [ ]:
#-- printando a matriz de confusão
confusion_matrix(y_test, y_pred)

In [ ]:
#-- plotando a matrix de confusão
_ = plot_confusion_matrix(logreg, X_test, y_test, cmap=plt.cm.Blues)

In [ ]:
#-- pritando as métricas de precisão e recall
print(classification_report(y_test, y_pred))

definições
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html

In [ ]:
#-- printando a ROC
_= plot_roc_curve(logreg, X_test, y_test)

### Utilizando as probabilidades

In [ ]:
#-- calculando a probabilidade dos teste
y_pred_proba = logreg.predict_proba(X_test)
y_pred_proba[:5]

In [ ]:
#-- pritando a probabilidade
y_pred_proba = y_pred_proba[:, 1]
y_pred_proba[:5]

In [ ]:
#-- comparando as respostas
preds_df = pd.DataFrame(data=[y_pred_proba, y_test.astype(str)],
                        index=['Prediction', 'True Value']).T

preds_df['Prediction'] = preds_df['Prediction'].astype(float)
preds_df['True Value'] = preds_df['True Value'].astype(str)

preds_df.head()

In [ ]:
teste = preds_df[preds_df['True Value'] == '0']
_ = sns.distplot(teste['Prediction'],  kde=False, label='Não Promov')
teste1 = preds_df[preds_df['True Value'] == '1']
_ = sns.distplot(teste1['Prediction'],  kde=False, label='Promov')
plt.legend(prop={'size': 12})
plt.title('Não Promovido X Promovido')
plt.xlabel('Predito')
plt.ylabel('Quantidade')

In [ ]:
#-- ajustando a probabilidade
y_pred_customizado = y_pred_proba >= 0.70

In [ ]:
#-- printando as novas métricas
print(classification_report(y_test, y_pred_customizado))

### Outras métricas populares

In [ ]:
print(f'auc (test): {roc_auc_score(y_test, y_pred_proba):.4f}')

In [ ]:
#-- métrica - qt menor melhor
print(f'log loss (test): {log_loss(y_test, y_pred_proba):.3f}')

### Identificação de overfitting

In [ ]:
y_pred_proba_train = logreg.predict_proba(X_train)[:, 1]

In [ ]:
#-- comparando os resultados do modelo de treino com o de teste
print(f'auc (train): {roc_auc_score(y_train, y_pred_proba_train):.4f}')
print(f'auc (test): {roc_auc_score(y_test, y_pred_proba):.4f}')

## Tentando melhorar o modelo

### One-hot-encoding

In [ ]:
#-- printando os nomes das colunas
df_ano.columns.values

In [ ]:
#-- selecionando as colunas do dataset
df_ano_misto = df_ano[['Tempo de Casa', 'Idade Atual/ Desligamento', 'Idade na Admissão', '2013/14 Goal Achievement', '2014/15 Goal Achievement', '2015/16 Goal Achievement', '2016/17 Goal Achievement', 'Estado Civil', 'Educação', 'Sexo', 'PROMOVIDO']].copy()

In [ ]:
#-- printando o head do dataset
df_ano_misto.head().T

In [ ]:
#-- removendo as colunas de 2013/14 e 2014/15
df_ano_misto.drop(columns=['2013/14 Goal Achievement', '2014/15 Goal Achievement'], inplace = True)

In [ ]:
#-- preenchendo os missing values
df_ano_misto.fillna(method='bfill', inplace=True)
df_ano_misto.fillna(method='ffill', inplace=True)

In [ ]:
#-- verificando se não existem mais missing values no dataset
df_ano_misto.isnull().sum()

In [ ]:
#-- transformando os dados discretos em dummies
pd.get_dummies(df_ano_misto['Estado Civil']).head()

In [ ]:
#-- aplicando a função dummies
df_dummies = pd.get_dummies(df_ano_misto, columns=['Estado Civil', 'Educação', 'Sexo'])

In [ ]:
#-- printando o head
df_dummies.head()

In [ ]:
#-- separando os datasets
X = df_dummies.drop(columns=['PROMOVIDO'])
y = df_dummies['PROMOVIDO']

#-- splitando em ds de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=0)

In [ ]:
#-- aplicando o algoritmo
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred  = logreg.predict(X_test)
y_pred_proba = logreg.predict_proba(X_test)[:, 1]

In [ ]:
#-- verificando a métrica log loss
print(f'log loss (test): {log_loss(y_test, y_pred):.3f}')

In [ ]:
#-- plotando a matrix de confusão
_ = plot_confusion_matrix(logreg, X_test, y_test, cmap=plt.cm.Blues)

In [ ]:
#-- printando as novas métricas
print(classification_report(y_test, y_pred))

### Tratamento de outliers

In [ ]:
#-- copiando o dataset
df_no_outliers = df_dummies.copy()

In [ ]:
#-- plotando os outliers
sns.boxplot(df_no_outliers['Tempo de Casa'])

In [ ]:
#-- plotando os outliers
sns.boxplot(df_no_outliers['Idade na Admissão'])

In [ ]:
#-- plotando os outliers
sns.boxplot(df_no_outliers['2015/16 Goal Achievement'])

In [ ]:
#-- plotando os outliers
sns.boxplot(df_no_outliers['2016/17 Goal Achievement'])

In [ ]:
#-- removendo os outliers
df_no_outliers = df_no_outliers[(df_no_outliers['2015/16 Goal Achievement'] > 88) & (df_no_outliers['2015/16 Goal Achievement'] < 120)]
df_no_outliers = df_no_outliers[(df_no_outliers['2016/17 Goal Achievement'] < 122) & (df_no_outliers['2016/17 Goal Achievement'] > 92)]

In [ ]:
#-- criando os ds
X = df_no_outliers.drop(columns=['PROMOVIDO'])
y = df_no_outliers['PROMOVIDO']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=0)

In [ ]:
#-- aplicando o algoritmo
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [ ]:
#-- plotando a matrix de confusão
_ = plot_confusion_matrix(logreg, X_test, y_test, cmap=plt.cm.Blues)

In [ ]:
#-- printando as novas métricas
print(classification_report(y_test, y_pred))

### Padronização (ou standardization) dos dados

In [ ]:
#-- copiando o dataset
df_standardized = df_dummies.copy()

In [ ]:
df_standardized.head().T

In [ ]:
#-- preparando os ds de análise
X = df_standardized.drop(columns=['PROMOVIDO'])
y = df_standardized['PROMOVIDO']

In [ ]:
#-- standarizando os dados
scaler = StandardScaler()
scaled_data = scaler.fit_transform(X)

X_scaled = pd.DataFrame(scaled_data, 
                        index=X.index,
                        columns=X.columns)

df_standardized = pd.concat([X_scaled, y], axis='columns')

In [ ]:
#-- pritando os dados do ds
X_scaled.head()

In [ ]:
#-- separando as amostras de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, 
                                                    random_state=0)

In [ ]:
#-- aplicando o algoritmo
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_proba = logreg.predict_proba(X_test)[:, 1]

In [ ]:
#-- pritando a métrica de teste
print(f'log loss (test): {log_loss(y_test, y_pred_proba):.3f}')

In [ ]:
#-- printando a ROC
_= plot_roc_curve(logreg, X_test, y_test)

### Seleção de features

In [ ]:
#-- parametrizando o algoritmo de seleção das variáveis
selector = RFECV(logreg, step=1, scoring='roc_auc', n_jobs=-1)

In [ ]:
#-- aplicando o algoritmo
_ = selector.fit(X_train, y_train)

In [ ]:
#-- apresentando a seleção
pd.DataFrame(data=selector.support_,
             columns=['support'],
             index=X.columns).T

In [ ]:
#-- apresentando a seleção
pd.DataFrame(data=selector.ranking_,
             columns=['support'],
             index=X.columns).T

In [ ]:
#-- criando um novo dataset
df_selection = selector.transform(X_train)
df_selection.shape

In [ ]:
#-- selecionando a coluna de resposta
y_pred_proba = selector.predict_proba(X_test)[:, 1]

In [ ]:
#-- pritando a métrica de log loss
print(f'log loss (test): {log_loss(y_test, y_pred_proba):.3f}')

In [ ]:
#-- printando a ROC
_= plot_roc_curve(logreg, X_test, y_test)

In [ ]:
#-- checking multcolinearity
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

In [ ]:
#-- sorting data by multicolineatiry
vif.round(1).sort_values(['VIF Factor'], ascending=True)

### Balanceamento

In [ ]:
#-- o moodelo está balanceado?
pd.DataFrame(y)['PROMOVIDO'].value_counts()

In [ ]:
#-- criando o balanceamento
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=0)
columns = X_train.columns
os_data_X,os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns)
os_data_y = pd.DataFrame(data=os_data_y)

#-- checando
print("tamanho do dataset ",len(os_data_X))
print("número de clientes com cancelamento",len(os_data_y[os_data_y['PROMOVIDO']==0]))
print("número de clientes sem cancelamento",len(os_data_y[os_data_y['PROMOVIDO']==1]))
print("proporção de com cancelamento ",len(os_data_y[os_data_y['PROMOVIDO']==0])/len(os_data_X))
print("proporção de sem cancelamento ",len(os_data_y[os_data_y['PROMOVIDO']==1])/len(os_data_X))

In [ ]:
logreg_up = LogisticRegression()

In [ ]:
logreg_up.fit(os_data_X, os_data_y)
y_pred = logreg_up.predict(X_test)

In [ ]:
accuracy_test = accuracy_score(y_test, y_pred)
print(f'accuracy (test): {accuracy_test*100:.1f}%')

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
#-- printando a matriz de confusão
_ = plot_confusion_matrix(logreg_up, X_test, y_test, cmap=plt.cm.Blues)

In [ ]:
#-- printando as novas métricas
print(classification_report(y_test, y_pred))

## Salvando o modelo

Vamos salvar o modelo para conseguirmos carregá-lo em análises futuras pós-aula.
Para detalhes, veja a documentação do scikit-learn: [Model Persistence](http://scikit-learn.org/stable/modules/model_persistence.html).

In [ ]:
_ = joblib.dump(logreg, '../models/logreg.pkl') 

Vamos também salvar o dataset transformado, assim como foi utilizado pelo modelo final. Para facilitar a interpretação do modelo, salvaremos também uma versão para "display", que é sua versão antes da padronização dos dados.

In [ ]:
df_standardized.to_csv('../data/Base Analytics Processed.csv', index=False)